# 01 Attribute Network and Make Scenario Pickle

In [36]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

import pickle

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.INFO)

## Remote I/O

In [5]:
root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_02')
output_dir = input_dir
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')

In [6]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-26 12:23:45, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:23:45, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso


## Data Reads

In [ ]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(
    link_file = link_file, 
    node_file = node_file, 
    shape_file = shape_file
)

In [43]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2020-10-26 12:49:07, INFO: Read in transit feed from: /Users/wsp/Documents/GitHub/travel-model-two-networks/data/processed/version_02
2020-10-26 12:49:07, INFO: Read in transit feed from: /Users/wsp/Documents/GitHub/travel-model-two-networks/data/processed/version_02
2020-10-26 12:49:07, INFO: Removing calendar.txt from transit network config because file not found
2020-10-26 12:49:07, INFO: Removing calendar.txt from transit network config because file not found
2020-10-26 12:49:07, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-10-26 12:49:07, INFO: Removing calendar_dates.txt from transit network config because file not found


ValueError: not enough values to unpack (expected 3, got 1)

## Attribute the Network

In [21]:
r_net = mtc.determine_number_of_lanes(
    roadway_network = roadway_net,
    parameters = parameters,
    network_variable = 'lanes'
)

r_net.links_df.lanes.value_counts()

2020-10-26 12:33:09, INFO: Determining number of lanes
2020-10-26 12:33:09, INFO: Determining number of lanes
2020-10-26 12:33:09, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:33:09, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:36:07, INFO: Finished determining number of lanes using variable: lanes
2020-10-26 12:36:07, INFO: Finished determining number of lanes using variable: lanes


1    1486397
2     104207
3      22455
4      14124
5       6504
6        929
7        134
8         18
9          1
Name: lanes, dtype: int64

In [39]:
r_net = mtc.calculate_facility_type(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'ft')

r_net.links_df.ft.value_counts()

2020-10-26 12:45:24, INFO: Calculating Facility Type
2020-10-26 12:45:24, INFO: Calculating Facility Type
2020-10-26 12:45:24, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:45:24, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:46:52, INFO: Finished calculating roadway class variable: ft
2020-10-26 12:46:52, INFO: Finished calculating roadway class variable: ft


99    890153
7     538666
6      96883
5      58344
4      38332
3       6365
2       3252
1       2774
Name: ft, dtype: int64

In [40]:
r_net = mtc.calculate_useclass(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'useclass')

r_net.links_df.useclass.value_counts()

2020-10-26 12:46:52, INFO: Determining useclass
2020-10-26 12:46:52, INFO: Determining useclass
2020-10-26 12:46:52, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:46:52, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:46:52, INFO: Variable 'useclass' updated for some links. Returning without overwriting for those links. Calculating for other links
2020-10-26 12:46:52, INFO: Variable 'useclass' updated for some links. Returning without overwriting for those links. Calculating for other links
2020-10-26 12:46:52, INFO: Calculating and adding roadway network variable: useclass
2020-10-26 12:46:52, INFO: Calculating and adding roadway network variable: useclass
2020-10-26 12:46:52, INFO: Finished determining transit-only variable: useclass
2020-10-26 12:46:52, INFO: Finished determining transit-only variable: useclass


0    1634769
Name: useclass, dtype: int64

In [41]:
r_net = mtc.calculate_assignable(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'assignable')

# r_net.links_df.assignable.value_counts()

2020-10-26 12:46:52, INFO: Determining assignable
2020-10-26 12:46:52, INFO: Determining assignable
2020-10-26 12:46:52, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:46:52, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:46:52, INFO: Variable 'assignable' already in network. Returning without overwriting.
2020-10-26 12:46:52, INFO: Variable 'assignable' already in network. Returning without overwriting.


In [42]:
r_net = mtc.calculate_transit(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'transit')

r_net.links_df.transit.value_counts()

2020-10-26 12:46:52, INFO: Determining assignable
2020-10-26 12:46:52, INFO: Determining assignable
2020-10-26 12:46:52, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:46:52, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/mtc-Lasso
2020-10-26 12:46:52, INFO: Calculating and adding roadway network variable: transit
2020-10-26 12:46:52, INFO: Calculating and adding roadway network variable: transit
2020-10-26 12:46:52, INFO: Finished determining transit-only variable: transit
2020-10-26 12:46:52, INFO: Finished determining transit-only variable: transit


0    1634030
1        739
Name: transit, dtype: int64

## Create a Scenario

In [44]:
# base_scenario = {"road_net": r_net, "transit_net": transit_net}
base_scenario = {"road_net": r_net}
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2020-10-26 12:49:22, INFO: Creating Scenario
2020-10-26 12:49:22, INFO: Creating Scenario


## Write to Disk

In [45]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_00.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))